# Лабораторная работа 2

## Задание 1
Реализовать в среде MATLAB метод наискорейшего спуска, сопряженных градиентов, Ньютона, правильного симплекса, циклического покоординатного спуска, Хука-Дживса и случайного поиска, при реализации методов использовать аналитические значения производных и их разностные аппроксимации.

In [1]:
import math

import numpy as np
from sympy import lambdify
from sympy.abc import x, y

from functools import total_ordering


@total_ordering
class Point:
    """Класс точки функции и ее значения для более удобных манипуляций"""

    def __init__(self, x_p, value=float('inf')):
        self.x_point = x_p
        self.value = value

    def __eq__(self, other):
        return self.value == other.value

    def __le__(self, other):
            return self.value <= other.value
    
    def get_coords(self):
         return self.x_point


@total_ordering
class Point2:
    """Класс точки функции и ее значения для более удобных манипуляций"""

    def __init__(self, x_p, y_p, value=float('inf')):
        self.x_point = x_p
        self.y_point = y_p
        self.value = value

    def __eq__(self, other):
        return self.value == other.value

    def __le__(self, other):
            return self.value <= other.value
    
    def get_coords(self):
         return np.array([self.x_point, self.y_point])


def calcule_gradient(func, symbols, values) -> tuple[np.ndarray, int]:

    direvatives_matrix = [func.diff(arg) for arg in symbols]
    gm_k1 = np.array([lambdify(symbols, derevat)(*values) for derevat in direvatives_matrix])
    direvatives_calculatus = len(direvatives_matrix)

    return gm_k1, direvatives_calculatus


def hessian(func, symbols, values) -> np.ndarray:

    total_calculates = 0
    hessian = np.empty([len(symbols)] * 2)

    for row in range(hessian.ndim):
        for col in range(hessian.ndim):
            calculated_direvative = func.diff(symbols[col], symbols[row])
            total_calculates += 2

            hessian[row][col] = lambdify(symbols, calculated_direvative)(*values)
            total_calculates += 1
    
    return hessian, total_calculates


def bitByBitSearch(func, x, eps=10e-3):
    """Реализация метода поразрядного поиска. Возвращает точку минимума, значение минимума функции и количество вычислений"""

    cur_eps = 0.25 if 0.25 > eps else eps

    direction_to_right = True
    compute_number = 0
    
    func_arg = x
    last_point = Point(func_arg, func(func_arg))

    
    while True:

        while True:
            
            func_arg += cur_eps if direction_to_right else -1*cur_eps
            
            func_value = func(func_arg)
            func_point = Point(func_arg, func_value)
            compute_number += 1
            
            if func_point >= last_point:
                break
            else:
                last_point = func_point

        if cur_eps <= eps:
            break
        else:
            last_point = func_point
            direction_to_right = not direction_to_right
            cur_eps /= 4

    return last_point.get_coords(), last_point.value, compute_number


In [2]:
def fastest_downhill(func, x_val, y_val, eps):

    iteration_number = 0
    calcul_number = 0
    point = Point2(x_val, y_val)

    func_lambd = lambdify([x, y], func)
    point.value = func_lambd(x_val, y_val)

    while True:
        iteration_number += 1

        gradient_matrix, difs_count = calcule_gradient(func, [x, y], point.get_coords())
        calcul_number += difs_count
        gradient_norm: float = np.linalg.norm(gradient_matrix)

        if gradient_norm < eps:
            break

        x_k = point.get_coords()
        F_min = lambda alpha: func_lambd(*(x_k - alpha * gradient_matrix))
        alpha_min, new_value, func_count = bitByBitSearch(F_min, 0)
        calcul_number += func_count

        x_new, y_new = x_k - alpha_min * gradient_matrix
        point = Point2(x_new, y_new, new_value)

    return point.get_coords(), point.value, iteration_number, calcul_number

In [3]:
def conjugate_gradients_method(func, x_val, y_val, eps):

    k = 0
    iteration_count = 0
    calcul_number = 0
    point = Point2(x_val, y_val)
    func_lambd = lambdify([x, y], func)

    gm_k1, difs_count = calcule_gradient(func, [x, y], point.get_coords())
    calcul_number += difs_count

    p_0 = -gm_k1

    while True:
        iteration_count += 1

        F_min = lambda alpha: func_lambd(*(point.get_coords() + alpha * p_0))
        alpha_min, _, func_count = bitByBitSearch(F_min, 0)
        calcul_number += func_count

        x_k2 = point.get_coords() + alpha_min * p_0
        gm_k2, difs_count = calcule_gradient(func, [x, y], x_k2)
        calcul_number += difs_count

        gm_k2_norm = np.linalg.norm(gm_k2)
        if gm_k2_norm < eps:
            break

        if k + 1 == 2:
            beta = 0
            k = 0
        else:
            beta = gm_k2_norm**2 / np.linalg.norm(gm_k1)**2
            k += 1

        p_0 = -gm_k2 + beta * p_0
        point = Point2(*x_k2)
    
    return x_k2, func_lambd(*x_k2), iteration_count, calcul_number + 1

In [4]:
def cool_newton_method(func, x_val, y_val, eps):

    iteration_count = 0
    calcul_number = 0
    point = Point2(x_val, y_val)
    func_lambd = lambdify([x, y], func)

    while True:
        gm_k1, difs_count = calcule_gradient(func, [x, y], point.get_coords())
        calcul_number += difs_count

        if np.linalg.norm(gm_k1) < eps:
            break
        iteration_count += 1

        func_hessian, hessian_count = hessian(func, [x, y], point.get_coords())
        calcul_number += hessian_count

        inv_hessian = np.linalg.inv(func_hessian)

        x_next = point.get_coords() - np.matmul(inv_hessian, gm_k1)
        point = Point2(*x_next)
    
    return point.get_coords(), func_lambd(*point.get_coords()), iteration_count, calcul_number + 1


In [10]:
def simplex_method(f, x_start, tol=1e-6):
    alpha = 1  # коэффициент отражения
    sigma = 0.5  # коэффициент уменьшения симплекса
    max_iter = 500
    
    n = len(x_start)  # размерность пространства (2 для двумерного случая)
    simplex = np.zeros((n + 1, n))  # создаем начальный симплекс
    simplex[0] = x_start

    # Начальный симплекс создается путем добавления небольших смещений к начальной точке
    for i in range(1, n + 1):
        x = np.copy(x_start)
        x[i - 1] += 0.05  # небольшое смещение по одной из координат
        simplex[i] = x
    
    # f_values = np.apply_along_axis(f, 1, simplex)  # вычисляем значения функции в точках симплекса
    f_values = np.array(list(map(lambda sim_point: f(*sim_point), simplex)))
    iter_count = 0
    
    while iter_count < max_iter:
        iter_count += 1
        
        # Сортируем вершины симплекса по значениям функции
        idx = np.argsort(f_values)
        simplex = simplex[idx]
        f_values = f_values[idx]
        
        # Проверяем условие сходимости
        if np.std(f_values) < tol:
            break
        
        # Определяем центроид (среднее всех точек, кроме худшей)
        centroid = np.mean(simplex[:-1], axis=0)
        
        # Отражение худшей точки
        xr = centroid + alpha * (centroid - simplex[-1])
        fr = f(*xr)
        
        if fr < f_values[-1]:
            simplex[-1] = xr
            f_values[-1] = fr
            continue
        
        # Если сжатие не сработало, уменьшаем симплекс
        simplex[1:] = simplex[0] + sigma * (simplex[1:] - simplex[0])
        f_values[1:] = np.array(list(map(lambda sim_point: f(*sim_point), simplex[1:])))
    
    return simplex[0], f_values[0], iter_count, 0


In [11]:
research_func = 4*x**2 + 3*y**2 - 4*x*y + x

minimum_p = simplex_method(lambdify([x, y], research_func), [.0, .0], 0.0001)
print(f"Minimum point --- func({minimum_p[0]}) = {minimum_p[1]}; {minimum_p[2]} iterations and {minimum_p[3]} calculates")

Minimum point --- func([-0.18125 -0.1125 ]) = -0.0934375; 33 iterations and 0 calculates
